In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## Importing library
import tensorflow as tf
from google.colab import files
import numpy as np
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
import cv2

In [16]:
## Final model Class define
class Final_Model:

    def __init__(self):
        """
        Final_Model constructer
        """
        self.model = self.get_model()
        self.model.load_weights("/content/drive/MyDrive/Colab Notebooks/AAIC/2nd case study/Models/RESNET50.h5")
    
    def predict(self,image):
        """
        This function return predicted value

        Input:
            image: 320x320x3 size array
        Output:
            Pred: predicted dimensions (x_center,y_center,box oriantation,box height,box width) 
            Edges: 4x2 array, contains points of corner
        """

        height, width = image.shape[:2]
        tt,ttt =  1,1
        T = np.float32([[1, 0, 50 * tt], [0, 1, 50 * ttt]])
        image = cv2.warpAffine(image, T, (width, height))

        pred = self.model.predict(np.array([image]))[0]
        edges = self.grasp_to_bbox_pred(pred[0], pred[1],pred[2],pred[3],pred[4]) - np.array([50,50])

        return pred, edges

    def grasp_to_bbox_pred(self,x, y, angle, h, w):
        """
        This function return corner points using given dimensions (x_center,y_center,box oriantation,box height,box width) 

        Input:
            x: x cordinate of box center
            y: y cordinate of box center
            angle: Oriantation of box respect to positive x-axis
            h: height of box.
            w: width of box
        Output:
            Edges: 4x2 array, contains points of corner
        """
        edge1 = (x - w/2, y + h/2)
        edge2 = (x + w/2, y + h/2)
        edge3 = (x + w/2, y - h/2)
        edge4 = (x - w/2, y - h/2)
        a = np.array([edge1, edge2, edge3, edge4])
        a = a - (x,y)
        angle = angle * (3.14/180)
        fun = lambda x: (x[:,0] * np.cos(angle) - x[:,1] * np.sin(angle), x[:,0] * np.sin(angle) + x[:,1] * np.cos(angle))
        a = fun(a)
        a = np.array(a).T
        a = a + (x,y)
        return a

    def plot_predict(self,image):
        """
        This function return predicted values and plot predicted values on image

        Input:
            image: 320x320x3 size array
        Output:
            Pred: predicted dimensions (x_center,y_center,box oriantation,box height,box width) 
            Edges: 4x2 array, contains points of corner
        """
        plot_im = image

        height, width = image.shape[:2]
        tt,ttt =  1,1
        T = np.float32([[1, 0, 50 * tt], [0, 1, 50 * ttt]])
        image = cv2.warpAffine(image, T, (width, height))

        pred = self.model.predict(np.array([image]))[0]
        edges = self.grasp_to_bbox_pred(pred[0], pred[1],pred[2],pred[3],pred[4])

        x = edges[:,0].reshape(1,-1)[0]
        y = edges[:,1].reshape(1,-1)[0]
        
        x = np.concatenate([x,x[:1]]) - 50
        y = np.concatenate([y,y[:1]]) - 50
        

        fig, ax = plt.subplots(figsize = (8,8))
        ax.imshow(plot_im)
        ax.plot(x,y, 'ro-',color = 'green')

        return pred,edges


    def get_model(self,lr = 0.001):

        """
        This function load trained model and return it.

        Input: 
            lr: learning rate but its not require here.
        Output:
            model: return trained model, type: tf.keras.Model
        """

        def resenet_load():
            RESNET50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',input_shape=(320,320,3), classes=2048)
            for i in range(len(RESNET50.layers)):
                RESNET50.layers[i].trainable = False
            return RESNET50

        RESNET50 = resenet_load()

        input = tf.keras.layers.Input((320,320,3,))
        resnet = RESNET50(input)
        x = tf.keras.layers.Flatten()(resnet)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        output = tf.keras.layers.Dense(5)(x)
        model = tf.keras.models.Model(input,output)
    
        model.compile(tf.keras.optimizers.Adam(lr), loss = 'mse')
        return model


In [17]:
model = Final_Model()

In [33]:
# uploaded = files.upload()

# for filename in uploaded.keys():
#     im = np.array(Image.open(BytesIO(uploaded[filename])))[:,:,:3]
#     (x, y, angle, h, w),edges = model.plot_predict(im)

In [11]:
import glob
images = glob.glob("/content/*.png")

In [31]:
## Precting random 140 images
fig,ax = plt.subplots(20,7,figsize = (20,60))
k = 0
for i in range(20):
    for j in range(7):

        im = cv2.imread(images[k])
        (x, y, angle, h, w),edges = model.predict(im)
        x = edges[:,0].reshape(1,-1)[0]
        y = edges[:,1].reshape(1,-1)[0]
        
        x = np.concatenate([x,x[:1]])
        y = np.concatenate([y,y[:1]])
        
        ax[i,j].imshow(im)
        ax[i,j].plot(x,y, 'ro-',color = 'green')

        ax[i,j].set_xticklabels([])
        ax[i,j].set_yticklabels([])
        ax[i,j].set_aspect('equal')

        k+=1
fig.subplots_adjust(wspace=0, hspace=0)

Output hidden; open in https://colab.research.google.com to view.

## Refrences:
* https://www.researchgate.net/publication/310953119
* https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet
* https://github.com/tnikolla/robot-grasp-detection
* https://www.appliedaicourse.com/